In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
if os.getcwd()[-8:]=='examples':
    os.chdir('..')
    
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Possible GPUS

In [2]:
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image

from opacus import PrivacyEngine
from opacus.validators import ModuleValidator
import torchdefenses as td
import torchdefenses.trainer as tr
from torchdefenses.transforms.cutout import Cutout

import src
from src.utils import *

### Settings

In [3]:
#### FOR TRAINING
EPOCHS = 200
BATCH_SIZE = 64

#### FOR MODELING
MODEL_NAME = "WRN16-4_WS"
DATA = "CIFAR100"
MEAN = [0.5071, 0.4867, 0.4408]
STD = [0.2675, 0.2565, 0.2761]
N_CLASSES = 100
LR = 0.1
RHO = 0.1

#### PATH SAVING
PATH = "./saved/"
NAME = "cifar-100-warmup"
SAVE_PATH = PATH + NAME

### Data loader

In [4]:
## FOR EDM
images = torch.load('./data/cifar-100-edm/cifar100_data_sampled.pt').type('torch.FloatTensor')
labels = torch.load('./data/cifar-100-edm/cifar100_data_sampled_labels.pt').type('torch.LongTensor')

train_transform  = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.Normalize(MEAN, STD),
                    Cutout(),
                   ])

In [5]:
dataset = PtDataset(images, labels, transform=train_transform)
public_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, 
                           drop_last=False, num_workers=16, pin_memory=True)

data = td.Datasets(data_name=DATA,
                   train_transform = transforms.Compose([
                    transforms.RandomCrop(32, padding=4),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                    Cutout(),
                   ]),
                   val_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                   ]),
                   test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD),
                   ]))
_, test_loader = data.get_loader(batch_size=BATCH_SIZE, drop_last_train=False, num_workers=16)

Files already downloaded and verified
Files already downloaded and verified
Data Loaded!
Train Data Length : 50000
Test Data Length : 10000


### Model & Optimizer

In [6]:
#### Load model
model = src.utils.load_model(model_name=MODEL_NAME, n_classes=N_CLASSES).cuda() # Load model
model = ModuleValidator.fix(model)
set_groupnorm_num_groups(model, num_groups=16)

ModuleValidator.validate(model, strict=False)

pytorch_total_params = sum(p.numel() for p in model.parameters())

print("model params: {:.4f}M".format(pytorch_total_params/1000000))

rmodel = td.RobModel(model, n_classes=N_CLASSES, normalization_used={'mean':MEAN, 'std':STD})

WRN16-4_WS is loaded.
model params: 2.7720M


### Start Training

In [7]:
trainer = tr.Standard(rmodel)
trainer.setup(optimizer=f"SGD(lr={LR}, momentum=0.9, weight_decay=0.0005)",
              scheduler="Cosine", scheduler_type="Epoch",
              minimizer=f"SAM(rho={RHO})",
              n_epochs=EPOCHS, n_iters=len(public_loader)
             )
trainer.record_rob(public_loader, test_loader, eps=None)
trainer.fit(train_loader=public_loader, n_epochs=EPOCHS,
            save_path=SAVE_PATH, save_best={"Clean(Val)":"HBO"},
            save_type=None, save_overwrite=True, record_type="Epoch")

05/30/2024 21:52:28:WARNING:Save file(s) will be overwritten:./saved/cifar-100-warmup
05/30/2024 21:52:28:WARNING:Save file(s) will be overwritten:./saved/cifar-100-warmup/init.pth
05/30/2024 21:52:28:WARNING:Save file(s) will be overwritten:./saved/cifar-100-warmup/last.pth
05/30/2024 21:52:28:WARNING:Save file(s) will be overwritten:./saved/cifar-100-warmup/log.txt


[Standard]
Training Information.
-Epochs: 200
-Optimizer: SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    initial_lr: 0.1
    lr: 0.1
    maximize: False
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0005
)
-Scheduler: <torch.optim.lr_scheduler.CosineAnnealingLR object at 0x7f42bcf19ca0>
-Minmizer: <torchdefenses.optim.minimizer.SAM object at 0x7f42bcf19370>
-Save Path: ./saved/cifar-100-warmup
-Save Type: None
-Record Type: Epoch
-Device: cuda:0
-----------------------------------------------------------------------
Epoch   CALoss   CALoss_1   Clean(Tr)   Clean(Val)   lr       s/it     
1       4.0166   4.2469     12.2580     7.1200       0.1000   0.0807   
-----------------------------------------------------------------------
2       2.4448   2.7867     31.7400     12.8100      0.1000   0.0784   
-----------------------------------------------------------------------
3       1.6125   2.0507     60.0440     17.5200      0.1000   0.

### Evaluation

In [8]:
rmodel.load_dict(PATH+NAME+'/last.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 200), ('Iter', 782), ('CALoss', 0.003489785362035036), ('CALoss_1', 0.0482649952173233), ('Clean(Tr)', 99.996), ('Clean(Val)', 36.03), ('lr', 6.168375916970619e-06), ('s/it', 0.07906209846547314)])


36.03

In [9]:
rmodel.load_dict(PATH+NAME+'/best.pth')
rmodel.eval_accuracy(test_loader)

Model loaded.
Record Info:
OrderedDict([('Epoch', 189), ('Iter', 782), ('CALoss', 0.0029403616208583117), ('CALoss_1', 0.04140426218509674), ('Clean(Tr)', 100.0), ('Clean(Val)', 36.19), ('lr', 0.0008856374635655645), ('s/it', 0.08021366368286445)])


36.19